# Install all the necessary libraries for running the scripts

In [1]:
!pip install pandas
!pip install numpy


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python3.10 -m pip install --upgrade pip


# Adding in the 3 CSV Files for the Matching
1. Patient Data taken from Stony Brook University Hospital and Johns Hopkins University Hospital
2. Zip code to County crosswalk file provided by the U.S. Office of Policy Development and Research (PD&R) is used to add county names to the addresses. The crosswalk file is available at https://www.huduser.gov/apps/public/uspscrosswalk/home
3. The SVI data is from the Center for Disease Control and Prevention (CDC) and is available at https://www.atsdr.cdc.gov/placeandhealth/svi/data_documentation_download.html (**Year**: 2020; **Geography**: United States; **Geography Type**: Counties)

In [2]:
import pandas as pd

# Load the three CSV files into dataframes
address_data = pd.read_csv('data/cleaned_sample_address_data.csv')
zip_county_crosswalk = pd.read_csv('data/ZIP_COUNTY_032023.csv')
svi_data = pd.read_csv('data/SVI_2020_US_county.csv')

# Match the ZIP CODES to County FIPS

In [3]:
# Pivot the ZIP_COUNTY crosswalk to get each ZIP and its corresponding county FIPS in a single row
zip_county_pivot = zip_county_crosswalk.pivot_table(index='ZIP', values='COUNTY', aggfunc=lambda x: list(x)).reset_index()


In [4]:
# Convert the list of COUNTY FIPS to separate columns
max_counties_for_zip = zip_county_pivot['COUNTY'].apply(len).max()
for i in range(max_counties_for_zip):
    zip_county_pivot[f'COUNTY_{i+1}'] = zip_county_pivot['COUNTY'].apply(lambda x: x[i] if i < len(x) else None)
zip_county_pivot = zip_county_pivot.drop(columns=['COUNTY'])

In [5]:
# Merge the address_data with the reshaped zip_county_pivot
merged_data = pd.merge(address_data, zip_county_pivot, left_on='ZIP CODE', right_on='ZIP', how='left')
merged_data = merged_data.drop(columns=['ZIP'])

# Pull the National Overall SVI Score ('RLP_THEMES') for each County FIPS

In [6]:
# Create a dictionary to map STCNTY (county FIPS) to RPL_THEMES
fips_to_rpl = dict(zip(svi_data['STCNTY'], svi_data['RPL_THEMES']))

# For each COUNTY column in the merged dataframe, map the RPL_THEMES value
for i in range(1, max_counties_for_zip + 1):
    merged_data[f'RPL_THEMES_{i}'] = merged_data[f'COUNTY_{i}'].map(fips_to_rpl)

# Save the final merged dataframe to a CSV file
merged_data.to_csv('data/final_merged_data.csv', index=False)

In [7]:
# Pull the Unique Value Count of the RPL_THEMES column
merged_data['RPL_THEMES_1'].value_counts()

RPL_THEMES_1
0.8845    1394
0.9955     876
0.6668     711
0.7747     692
0.5808     116
0.6817       4
Name: count, dtype: int64